# Tarea 2: Representación de Texto y Motores de Búsqueda

En esta tarea vamos a ver cómo implementar un motor de búsqueda básico utilizando técnicas de representación de texto y medidas de similitud. Vamos a aprender a usar algunas funciones simples de las librerías scikit-learn y pandas.

Primero, vamos a importar algunas librerías que necesitaremos antes de seguir adelante.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer  # Tf-idf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split  # Hablaremos más sobre validación y selección de modelos después.
from sklearn.naive_bayes import MultinomialNB  # Un clasificador simple, no hemos visto cómo funciona esto en clases.
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

Aquí te dejo una función de utilidad. Se utiliza para ordenar un arreglo numpy por su columna n-ésima (por defecto, ordena por la primera columna con n = 0). Devuelve los resultados en orden descendente. No es necesario que modifiques esta función, y no es obligatorio que la uses si no quieres.

In [2]:
def sort_array_col(arr, n=0):
    return arr[arr[:,n].argsort()[::-1]]

## Leyendo los datos

Tenemos un conjunto de resúmenes de artículos en un archivo CSV de tres temas diferentes (bioinformática, procesamiento de datos, redes). Tu primera tarea es leer el archivo CSV y extraer los artículos. Nos interesa solamente los resúmenes y el tema de cada artículo:

In [7]:
# Leer el archivo CSV 02_Dataset.csv con Pandas.
data = pd.read_csv("02_Dataset.csv")

# Convertir el tema en una variable numérica (pista: usar el llamado de función data.topic.map). 
# Asignar 0 a 'bioinformatics', 1 a 'network' y 2 a 'preprocessing data'.
data['topic_num'] = data['topic'].map({'bioinformatics': 0, 'network': 1, 'preprocessing data': 2})

X = data['abstract'].values
y = data['topic_num'].values

Vamos a dividir el conjunto de datos en un conjunto de entrenamiento (que utilizaremos para entrenar los modelos de representación) y un conjunto de pruebas que usaremos más adelante en la última parte de la tarea. Aunque no es estrictamente necesario para la primera parte de esta tarea, es una buena práctica para la segunda parte.

In [8]:
# Dividir X e y en conjuntos de entrenamiento y prueba
# 70% del conjunto de datos se usa para entrenar los modelos.
# En este caso solo utilizaremos este conjunto de datos para la primera parte.
# 30% del conjunto de datos se usa para probar los modelos. 
# Esto se usará para probar el clasificador de la última parte.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3, test_size=0.3)

## Modelo de Bag of Words (BoW)

Ahora construimos el modelo de Bag of Words (BoW).

In [ ]:
# Instanciar el vectorizador
# Para Bag of Words quieres usar CountVectorizer con los siguientes parámetros: 
# stop_words='english', ngram_range=(1, 2), max_df=0.9, min_df=2.
# Puedes intentar mejorar el rendimiento de tu modelo cambiando los últimos dos parámetros.
# Notar que estos modelos son específicos para este problema.
# En un problema real, los parámetros serían posiblemente muy diferentes.
vectorizador = CountVectorizer(stop_words='english', ngram_range=(1, 2), max_df=0.9, min_df=2)
vect_bow =
# Ajustar el modelo cn X_train.

# Luego, usalo para crear la matriz de documentos y términos.
X_train_dtm_bow = # Completar.

Ahora vectorizamos el conjunto de pruebas.

In [ ]:
# Transformar los datos de prueba (utilizando el vocabulario ajustado) en una matriz documento-término
X_test_dtm_bow = # Completar

¡Ahora vamos a consultar nuestro modelo! Probaremos el modelo con la cadena de búsqueda proporcionada en los datos de X_train. Siéntete libre de probar con tus propias cadenas, ¡es un ejercicio divertido! :)

Debes devolver los elementos ordenados por su similitud (los elementos más similares deben mostrarse primero) y el propio texto debe mostrarse también.

Solo debes devolver los elementos que tengan una similitud mayor a 0.

In [ ]:
query_string = "research on data science and biological analysis"
query_vec = vect_bow.transform([query_string])
# Hay que obtener la similitud del coseno entre la query y todos los documentos del conjunto de entrenamiento.
# Puedes implementar la similitud desde cero o usar librerías.
cos_sim = # Completar
# Puede que tengas que usar la función np.vstack
# Imprimir los documentos recuperados con similitud > 0 (posiblemente necesites usar sort_array_col).

## Modelo TF-IDF

Ahora construiremos el modelo TF-IDF.

In [ ]:
# Instanciar el vectorizador
# Para TF-IDF quieres usar TfidfVectorizer con los siguientes parámetros: 
# stop_words='english', ngram_range=(1, 2), max_df=0.95, min_df=2.
# Puedes tratar de mejorar el rendimiento cambiando los últimos dos parámetros.
vect_tf_idf = # Completar
# Ajustar el modelo y luego usarlo para crear una matriz documento-término
X_train_dtm_tf_idf = # Completar
# Transformar los datos de prueba (utilizando el vocabulario ajustado) en una matriz documento-término
X_test_dtm_tf_idf = # Completar

Ahora probemos nuestro nuevo modelo con la misma consulta.

In [ ]:
query_string = "research on data science and biological analysis"
query_vec = vect_bow.transform([query_string])
# Puedes reciclar tu código anterior!
# Hay que obtener la similitud del coseno entre la query y todos los documentos del conjunto de entrenamiento.
# Puedes implementar la similitud desde cero o usar librerías.
cos_sim = # Completar
# Puede que tengas que usar la función np.vstack
# Imprimir los documentos recuperados con similitud > 0 (posiblemente necesites usar sort_array_col).

## Nuestro Primer Clasificador

Ahora, vamos a saltarnos algunas clases (bueno, muchas clases) y te vamos a obligar a implementar un clasificador Bayesiano. ¿Espera, un qué? Básicamente, queremos un modelo que tome un texto y nos indique su tema. Esto podría hacerse utilizando el clasificador **Naïve Bayes**, que se implementa fácilmente en scikit-learn. Estudiaremos este método más adelante en el curso, pero por ahora lo aplicaremos como una caja negra.

Primero, probaremos un clasificador con el Modelo de Bag of Words (BoW).

In [ ]:
# La distribución multinomial normalmente requiere recuentos de características enteras.
# En este caso, lo usaremos con bag of words que nos entrega la cantidad de ocurrencias de cada palabra.
# Instanciar un modelo Naïve Bayes Multinomial
nb_bow = # Completar

# Entrenar el modelo usando X_train_dtm con la función fit.

# Hacer predicciones de clase para X_test_dtm.
y_pred_class = # Completar

# Calcular la precisión de las predicciones de clase utilizando accuracy_score de la librería metrics.

# Imprimir la matriz de confusión (¡veremos qué es esto pronto!) con la función confusion_matrix de la librería metrics.

Ahora probaremos un clasificador con el modelo TF-IDF.

In [ ]:
# La distribución multinomial normalmente requiere recuentos de características enteras.
# Sin embargo, en la práctica, recuentos fraccionales como tf-idf también pueden funcionar.
# Instanciar un modelo Naïve Bayes Multinomial
nb_tf_idf = # Completar

# Entrenar el modelo usando X_train_dtm_tf_idf con la función fit.

# Hacer predicciones de clase para X_test_dtm_tf_idf.
y_pred_class = # Completar

# Calcular la precisión de las predicciones de clase utilizando accuracy_score de la librería metrics.

# Imprimir la matriz de confusión (¡veremos qué es esto pronto!) con la función confusion_matrix de la librería metrics.

## Análisis

Entonces, ¿qué modelo es mejor? ¿Coincide esto con los resultados esperados de la clase? Modifica esta celda markdown y agrega tus respuestas a continuación. Considera el rendimiento tanto en la tarea de clasificación como en el modelo de motor de búsqueda. Prueba con consultas adicionales para ver cómo funcionan los modelos. Esta parte, por supuesto, será evaluada.

#### Respuesta:
- Inserta tu respuesta aquí.

## ¡Felicitaciones, has completado tu segunda tarea!